# Table of Contents

* [Environment and Configuration Setup - PowerShell](#setup-configuration)
* [Set Variables for HTTP Request Cells](#setup-http-configuration)
* [Data Holder APIs](#data_holder_apis)    
    * [Get JWKS](#get_jwks)
    * [Get Data Recipients](#get_datarecipients)
    * [Get Data Recipients Statuses](#get_datarecipients_status)
    * [Get Software Products Statuses](#get_sp_status)
    * [Get Data Holder Statuses](#get_dh_status)
    * [Get OpenId Provider Config - JWKS](#get_openid_jwks)
    * [Get OpenId Provider Config](#get_openid)
* [Data Recipient APIs](#data_recipient_apis)
    * [Get OpenId Provider Config](#get_openid)
    * [mTLS APIs]()
        * [Get Access Token](#get_token)
        * [Get Data Holder Brands](#get-data-holder-brands)
        * [Get Software Statement Assertion (SSA)](#get_ssa)
* [Register Administration APIs](#admin_apis)
    * [Get Metadata](#get_meta_data)
    * [Load Metadata](#load_meta_data)
    * [Add or Update Data Recipients](#update_data_recipients)
    * [Add or Update Data Holders](#update_data_holders)
    * [Get Identity Provider JWKS](#get_jwks_infosec)

## Environment and Configuration Setup <a class="anchor" id="setup-configuration"></a>

Set the required configuration variables which are reused by all the PowerShell cells in this notebook. 

**Note:** This cell must be executed prior to executing any PowerShell cells in this notebook.

In [ ]:
# Set Urls to use
$hostname = "localhost"
$mockRegisterBaseUrl = "https://${hostname}:7000"
$mockRegisterSecureBaseUrl = "https://${hostname}:7001"

# Set Data Reciepient variables
$mockDataRecipientCertificatePath = "..\\..\\CertificateManagement\\mtls\\client.pfx"
$mockDataRecipientCertificatePassword = "#M0ckDataRecipient#"
$industry = "Banking"
$dataRecipientBrandId = "F3F0C40B-9DF8-491A-AF1D-81CB9AB5F021"
$softwareProductId = "6F7A1B8E-8799-48A8-9011-E3920391F713"
$mdrClientAssertionUri = "https://${hostname}:7006/loopback/MockDataRecipientClientAssertion?iss=$softwareProductId"

# Load the Data Reciepient certificate with password
$certPassword = ConvertTo-SecureString -String $mockDataRecipientCertificatePassword -Force -AsPlainText
$mockDataRecipientCert = New-Object System.Security.Cryptography.X509Certificates.X509Certificate2 -ArgumentList $mockDataRecipientCertificatePath, $certPassword

<a class="anchor" id="setup-http-configuration"></a>
Set the required variables from above PowerShell cell to make them available for HTTP Request cells.

**Note:** This cell must be executed prior to executing any HTTP Request cells in this notebook.

In [ ]:
#!set --value @pwsh:hostname --name hostname
#!set --value @pwsh:mockRegisterBaseUrl --name mockRegisterBaseUrl
#!set --value @pwsh:softwareProductId --name softwareProductId
#!set --value @pwsh:industry --name industry

# Data Holder APIs <a class="anchor" id="data_holder_apis"></a>

The Mock Register's Data Holder APIs are the used by Data Holders. These APIs do not require any authentication.

## Get JWKS <a class="anchor" id="get_jwks"></a>


`GET /cdr-register/v1/jwks`

JWKS endpoint containing the public keys used by the CDR Register to validate the signature of issued SSAs and authenticate outbound calls to participants in the CDR.

In [ ]:
GET {{mockRegisterBaseUrl}}/cdr-register/v1/jwks

## Get Data Recipients <a class="anchor" id="get_datarecipients"></a>

`GET /cdr-register/v1/{industry}/data-recipients`

Endpoint used by participants to discover data recipients and associated brands and software products, available in the CDR ecosystem.

In [ ]:
GET {{mockRegisterBaseUrl}}/cdr-register/v1/{{industry}}/data-recipients
x-v:3

## Get Data Recipients Statuses<a class="anchor" id="get_datarecipients_status"></a>

`GET /cdr-register/v1/{industry}/data-recipients/status`

Endpoint used by participants to discover the statuses for Data Recipients from the CDR Register.

In [ ]:
GET {{mockRegisterBaseUrl}}/cdr-register/v1/{{industry}}/data-recipients/status
x-v: 2

## Get Software Products Statuses<a class="anchor" id="get_sp_status"></a>

`GET /cdr-register/v1/{industry}/data-recipients/brands/software-products/status`

Endpoint used by participants to discover the statuses for software products from the CDR Register.

In [ ]:
GET {{mockRegisterBaseUrl}}/cdr-register/v1/{{industry}}/data-recipients/brands/software-products/status
x-v: 2

## Get Data Holder Statuses<a class="anchor" id="get_dh_status"></a>

`GET /cdr-register/v1/{industry}/data-holders/status`

Endpoint used by participants to discover the statuses for Data Holders from the CDR Register

In [ ]:
GET {{mockRegisterBaseUrl}}/cdr-register/v1/{{industry}}/data-holders/status
x-v: 1

# Data Recipient APIs <a class="anchor" id="data_recipient_apis"></a>

The Mock Register's Data Recipient APIs are the used by Data Recipients. An access token is required to access the Get Data Holder Brands and Get Software Statement Assertion (SSA) APIs.

## Get OpenId Provider Config <a class="anchor" id="get_openid"></a>

`GET /idp/.well-known/openid-configuration`

Endpoint used by participants to discover the CDR Register OpenID configuration and obtain information needed to interact with it, including its OAuth 2.0 endpoint locations.

In [ ]:
GET {{mockRegisterBaseUrl}}/idp/.well-known/openid-configuration

## Get Access Token <a class="anchor" id="get_token"></a>

`GET /idp/connect/token`

Get an access token from the Register.  An access token is required to access the Get Data Holder Brands and Get Software Statement Assertion (SSA) APIs.

In [ ]:
# Make the request to get mdr client assertion on loopback Url
$response = Invoke-WebRequest -Uri $mdrClientAssertionUri -Certificate $mockDataRecipientCert
$mdrClientAssertion = $response.Content

# Define your parameters
$parameters = @{
    "grant_type" = "client_credentials"
    "client_id" = $softwareProductId
    "client_assertion_type" = "urn:ietf:params:oauth:client-assertion-type:jwt-bearer"
    "client_assertion" = $mdrClientAssertion
    "scope" = "cdr-register:read"
}

# Make the request
$response = Invoke-RestMethod -Uri "${mockRegisterSecureBaseUrl}/idp/connect/token"  `
                -Method Post -StatusCodeVariable responseCode `
                -Body $parameters -ContentType "application/x-www-form-urlencoded" `
                -Certificate $mockDataRecipientCert

$accessToken = $response.access_token

# Write the response status code and content to the console
Write-Host "Response Code: $responseCode"
Write-Host "Response Content: $accessToken"


## Get Data Holder Brands <a class="anchor" id="get-data-holder-brands"></a>


`GET /cdr-register/v1/{industry}/data-holders/brands`

Allows Data Recipients to discover Data Holder Brands available in the CDR ecosystem.

**Note:** This API requires an access token. The Get Access Token cell must be executed prior to executing this cell.

In [ ]:
# Set the API version and Authorization headers
$headers = @{
    "x-v"           = "2"
    "Authorization" = "Bearer $accessToken"
}

# Make the request
$res = Invoke-RestMethod -Uri "${mockRegisterSecureBaseUrl}/cdr-register/v1/all/data-holders/brands" `
                -Method Get -StatusCodeVariable resCode -Headers $headers -Certificate $mockDataRecipientCert `
                | ConvertTo-Json -Depth 10

# Write the response status code and content to the console
Write-Host "Response Code: $resCode"
Write-Host "Response Content: $($res)"

## Get Software Statement Assertion (SSA) <a class="anchor" id="get_ssa"></a>

`GET /cdr-register/v1/{industry}/data-recipients/brands/{dataRecipientBrandId}/software-products/{softwareProductId}/ssa`

Get a Software Statement Assertion (SSA) for a software product on the CDR Register to be used for Dynamic Client Registration with a Data Holder Brand.

**Note:** This API requires an access token. The Get Access Token cell must be executed prior to executing this cell.

In [ ]:
# Set the API version and Authorization headers
$headers = @{
    "x-v"           = "3"
    "Authorization" = "Bearer $accessToken"
}

# Make the request
$r = Invoke-RestMethod -Uri "${mockRegisterSecureBaseUrl}/cdr-register/v1/all/data-recipients/brands/${dataRecipientBrandId}/software-products/${softwareProductId}/ssa" `
                -Method Get -StatusCodeVariable rCode -Headers $headers -Certificate $mockDataRecipientCert `
                | ConvertTo-Json -Depth 10

# Write the response status code and content to the console
Write-Host "Response Code: $rCode"
Write-Host "Response Content: $($r)"

# Administration APIs <a class="anchor" id="admin_apis"></a>

The Mock Register's Administration APIs are used internally to add and update data in the Mock Register's database.

## Get Metadata <a class="anchor" id="get_meta_data"></a>

`GET /admin/metadata`

Get the metadata currently loaded in the Mock Register.

In [ ]:
GET https://{{hostname}}:7006/admin/metadata

## Load Metadata <a class="anchor" id="load_meta_data"></a>

`POST /admin/metadata`

Load new metadata in the Mock Register.

In [ ]:
# Load Payload from file. 
# Note: This payload is loaded from an external file instead of directly using a Polyglot cell due to it's large size. Large payload are difficult to read and modify in a Polyglot cell.
$metadataPayload = Get-Content -path .\payloads\metadataPayload.json -raw

# Make the request
$response = Invoke-RestMethod -Uri "https://${hostname}:7006/admin/metadata"  `
                -Method Post -StatusCodeVariable responseCode `
                -Body $metadataPayload `
                -ContentType "application/json" `

# Write the response status code and content to the console
Write-Host "Response Code: $responseCode"     
Write-Host "Response Content: $response"

## Add or Update Data Recipients <a class="anchor" id="update_data_recipients"></a>

`POST /admin/metadata/data-recipients`

Add or update a Data Recipient in the Mock Register.

In [ ]:
POST https://{{hostname}}:7006/admin/metadata/data-recipients 
x-v: 1
Content-Type: application/json
{
  "legalEntityId": "f9bde9a1-9fef-4bd9-b7ef-877369e076e3",
  "legalEntityName": "Mock Energy Company",
  "accreditationNumber": "ADR000001",
  "accreditationLevel": "UNRESTRICTED",
  "logoUri": "https://mocksoftware/img/Companylogo.png",
  "registrationNumber": null,
  "registrationDate": null,
  "registeredCountry": null,
  "abn": "34241177887",
  "acn": "241177887",
  "arbn": "342411088",
  "anzsicDivision": null,
  "organisationType": "COMPANY",
  "status": "ACTIVE",
  "dataRecipientBrands": [
    {
      "dataRecipientBrandId": "d50b74cf-e992-42fd-86d4-582ac9d72dbc",
      "brandName": "Mock Energy Tool",
      "logoUri": "https://mocksoftware/img/brandlogo.png",
      "status": "ACTIVE",
      "softwareProducts": [
        {
          "softwareProductId": "4754cf30-6cc2-4a9a-a9e1-4fb8f6d42121",
          "softwareProductName": "MyEnergyHelper",
          "softwareProductDescription": "A product to help you manage your energy costs",
          "logoUri": "https://mocksoftware/myenergyapp/img/logo.png",
          "sectorIdentifierUri": null,
          "clientUri": "https://mocksoftware/myenergyapp",
          "tosUri": "https://mocksoftware/myenergyapp/terms",
          "policyUri": "https://mocksoftware/myenergyapp/policy",
          "recipientBaseUri": "https://mock-data-recipient:9001",
          "revocationUri": "revocation",
          "redirectUris": [
            "consent/callback",
            "consent/callback2"
          ],
          "jwksUri": "jwks",
          "scope": "openid profile bank:accounts.basic:read bank:accounts.detail:read bank:transactions:read bank:payees:read bank:regular_payments:read common:customer.basic:read common:customer.detail:read cdr:registration cdr-register:read",
          "status": "ACTIVE",
          "certificates": [
            {
              "commonName": "MockDataRecipient",
              "thumbprint": "f0e5146a51f16e236844cf0353d791f11865e405"
            }
          ]
        }
      ]
    }
  ]
}

## Add or Update Data Holders <a class="anchor" id="update_data_holders"></a>

`POST /admin/metadata/data-holders`

Add or update a Data Holder in the Mock Register.

In [ ]:
POST https://{{hostname}}:7006/admin/metadata/data-holders HTTP/1.1
Content-Type: application/json
x-v: 1
{
  "dataHolderBrandId": "bb03be60-5c46-422e-a27e-aefa0015078d",
  "brandName": "Luna",
  "industries": [
    "banking"
  ],
  "logoUri": "https://lunabrand/img/logo.png",
  "status": "ACTIVE",
  "endpointDetail": {
    "version": "v1",
    "publicBaseUri": "https://publicapi.lunabank",
    "resourceBaseUri": "https://rb-api.lunabank",
    "infosecBaseUri": "https://isb-api.lunabank",
    "extensionBaseUri": "https://eb-api.lunabank",
    "websiteUri": "https://ws-api.lunabank"
  },
  "authDetails": {
    "registerUType": "SIGNED-JWT",
    "jwksEndpoint": "https://idp.lunabank/jwks"
  },
  "legalEntity": {
    "legalEntityId": "667407fc-dc7e-4be5-8795-5189240bcbca",
    "legalEntityName": "Luna Bank",
    "logoUri": "https://lunabank/img/logo.png",
    "registrationNumber": "20032023",
    "registrationDate": "2023-05-22",
    "registeredCountry": "AU",
    "abn": "12345678901",
    "acn": "345678901",
    "arbn": "345678901",
    "anzsicDivision": "123",
    "organisationType": "OTHER",
    "status": "ACTIVE"
  }
}

## Get Identity Provider JWKS <a class="anchor" id="get_jwks_infosec"></a>


`GET /idp/.well-known/openid-configuration/jwks`

JWKS endpoint containing the public keys used by the CDR Register's Identity Provider to validate access tokens used in Get Data Holder Brands and Get Software Statement Assertion (SSA) requests.

In [ ]:
GET {{mockRegisterBaseUrl}}/idp/.well-known/openid-configuration/jwks